# Postage Stamps

This notebook creates postage stamps for subtracted images generates by the Diference Image Analysis DIA pipeline. For instructions on running the pipeline, see this project's ReadMe.


## Setup

You will need to specify the directory of the differenced images you wish to process. This directory is the same as the directory autoimatically generated by the DIA pipeline.

In [1]:
# Input DIA data
diff_im_dir = '/global/u1/d/djp81/test_imdiff/'

# Output directory for postave stamps
postage_output_dir = './stamps'


In [2]:
import os

import numpy as np
from astropy.visualization import ZScaleInterval
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.table import Table, vstack
from astropy.wcs import WCS
from pathlib import Path

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
from desc_dc2_dm_data import REPOS

butler = dafPersist.Butler(diff_im_dir)


## Getting Target List

Subtraction results from the DIA pipeline are saved as pairs of *diffexp* and *diaSrc* files. The *diffexp* files contain the subracted image data while the *diaSrc* files act as a catlog of objects found in the subtracted image. We collect the paths of any DIA output files in the directory `diff_im_dir` specified above and create a combined table of detected objects.

In [3]:
def get_diff_file_paths(dia_dir):
    """Return lists of all diffexp and diaSrc file paths under ``dia_dir``
    
    Args:
        dia_dir (str): Path of DIA pipeline outputs
        
    Returns:
        A list of all diffexp paths as Path objects
        A list of all diaSrc paths as Path objects
    """

    diff_files, dia_files = [], []
    for filename in Path(dia_dir).glob('**/diffexp*.fits'):
        diff_files.append(filename)
        dia_files.append(Path(str(filename).replace('diffexp', 'diaSrc')))
        
    return diff_files, dia_files


diffexp_paths, diasrc_paths = get_diff_file_paths(diff_im_dir)

print('diffexp paths:')
print(diffexp_paths)

print('\ndiaSrc paths:')
print(diasrc_paths, '\n')

diffexp paths:
[PosixPath('/global/u1/d/djp81/test_imdiff/deepDiff/v00431306-fu/R10/diffexp_00431306-u-R10-S01-det028.fits')]

diaSrc paths:
[PosixPath('/global/u1/d/djp81/test_imdiff/deepDiff/v00431306-fu/R10/diaSrc_00431306-u-R10-S01-det028.fits')] 



Next we compile all of the diaSrc data into a single table

In [4]:
def read_diasrc_path(path):
    """Read a diaSrc file and return a table of detected sources
    
    Args:
        path (str): Path of the file to read
        
    Returns:
        An astropy table of source data, including coordinates
    """

    path_list = Path(path).stem.split('-')
    visit = int(path_list[0].lstrip('diaSrc_'))
    filter = path_list[1]
    raft = path_list[2]
    detector = int(path_list[-1].lstrip('det'))
    
    diaSrc_hdu_list = fits.open(path)
    diaSrc_data = diaSrc_hdu_list[1].data
    data_table = Table(np.array(diaSrc_data))
    data_table['visit'] = np.full(len(data_table), visit)
    data_table['filter'] = np.full(len(data_table), filter)
    data_table['raft'] = np.full(len(data_table), raft)
    data_table['detector'] = np.full(len(data_table), detector)
    return data_table
    

def read_diasrc_path_list(diasrc_list):
    """Read multiple diaSrc files and return a table of all detected sources
    
    Args:
        diasrc_list (list[str]): List of diaSrc paths to read
        
    Returns:
        An astropy table of all detected sources
    """
        
    diasrc_list = list(diasrc_list)  # Copy to avoid mutation
    data_table = read_diasrc_path(diasrc_list.pop())
    for path in diasrc_list:
        data_table = vstack(data_table, read_diasrc_path(path))
        
    return data_table

# First ten sources as an example
read_diasrc_path_list(diasrc_paths)[:10]


flags [10],id,coord_ra,coord_dec,parent,base_NaiveCentroid_x,base_NaiveCentroid_y,base_PeakCentroid_x,base_PeakCentroid_y,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,ip_diffim_NaiveDipoleCentroid_x,ip_diffim_NaiveDipoleCentroid_y,ip_diffim_NaiveDipoleCentroid_xErr,ip_diffim_NaiveDipoleCentroid_yErr,ip_diffim_NaiveDipoleCentroid_pos_x,ip_diffim_NaiveDipoleCentroid_pos_y,ip_diffim_NaiveDipoleCentroid_pos_xErr,ip_diffim_NaiveDipoleCentroid_pos_yErr,ip_diffim_NaiveDipoleCentroid_neg_x,ip_diffim_NaiveDipoleCentroid_neg_y,ip_diffim_NaiveDipoleCentroid_neg_xErr,ip_diffim_NaiveDipoleCentroid_neg_yErr,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,base_SdssShape_psf_yy,base_SdssShape_psf_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_12_0_instFlux,base_CircularApertureFlux_12_0_instFluxErr,base_CircularApertureFlux_17_0_instFlux,base_CircularApertureFlux_17_0_instFluxErr,base_CircularApertureFlux_25_0_instFlux,base_CircularApertureFlux_25_0_instFluxErr,base_CircularApertureFlux_35_0_instFlux,base_CircularApertureFlux_35_0_instFluxErr,base_CircularApertureFlux_50_0_instFlux,base_CircularApertureFlux_50_0_instFluxErr,base_CircularApertureFlux_70_0_instFlux,base_CircularApertureFlux_70_0_instFluxErr,base_GaussianFlux_instFlux,base_GaussianFlux_instFluxErr,base_PeakLikelihoodFlux_instFlux,base_PeakLikelihoodFlux_instFluxErr,base_PsfFlux_instFlux,base_PsfFlux_instFluxErr,base_PsfFlux_area,ip_diffim_NaiveDipoleFlux_pos_instFlux,ip_diffim_NaiveDipoleFlux_pos_instFluxErr,ip_diffim_NaiveDipoleFlux_neg_instFlux,ip_diffim_NaiveDipoleFlux_neg_instFluxErr,ip_diffim_NaiveDipoleFlux_npos,ip_diffim_NaiveDipoleFlux_nneg,ip_diffim_PsfDipoleFlux_pos_instFlux,ip_diffim_PsfDipoleFlux_pos_instFluxErr,ip_diffim_PsfDipoleFlux_neg_instFlux,ip_diffim_PsfDipoleFlux_neg_instFluxErr,ip_diffim_PsfDipoleFlux_chi2dof,ip_diffim_PsfDipoleFlux_pos_centroid_x,ip_diffim_PsfDipoleFlux_pos_centroid_y,ip_diffim_PsfDipoleFlux_pos_centroid_xErr,ip_diffim_PsfDipoleFlux_pos_centroid_yErr,ip_diffim_PsfDipoleFlux_neg_centroid_x,ip_diffim_PsfDipoleFlux_neg_centroid_y,ip_diffim_PsfDipoleFlux_neg_centroid_xErr,ip_diffim_PsfDipoleFlux_neg_centroid_yErr,ip_diffim_PsfDipoleFlux_centroid_x,ip_diffim_PsfDipoleFlux_centroid_y,ip_diffim_PsfDipoleFlux_centroid_xErr,ip_diffim_PsfDipoleFlux_centroid_yErr,ip_diffim_ClassificationDipole_value,ip_diffim_DipoleFit_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,ip_diffim_DipoleFit_pos_centroid_x,ip_diffim_DipoleFit_pos_centroid_y,ip_diffim_DipoleFit_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,ip_diffim_DipoleFit_neg_centroid_x,ip_diffim_DipoleFit_neg_centroid_y,ip_diffim_DipoleFit_centroid_x,ip_diffim_DipoleFit_centroid_y,ip_diffim_DipoleFit_instFlux,ip_diffim_DipoleFit_orientation,ip_diffim_DipoleFit_separation,ip_diffim_DipoleFit_chi2dof,ip_diffim_DipoleFit_signalToNoise,totFlux,totFluxErr,footprint,visit,filter,raft,detector
uint8,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,flo

## Creating Postage Stamps


Start by picking an examples source

In [5]:
# Read dia source data
diffexp_paths, diasrc_paths = get_diff_file_paths(diff_im_dir)
source_table = read_diasrc_path_list(diasrc_paths)
source = source_table[0]

source

flags [10],id,coord_ra,coord_dec,parent,base_NaiveCentroid_x,base_NaiveCentroid_y,base_PeakCentroid_x,base_PeakCentroid_y,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,ip_diffim_NaiveDipoleCentroid_x,ip_diffim_NaiveDipoleCentroid_y,ip_diffim_NaiveDipoleCentroid_xErr,ip_diffim_NaiveDipoleCentroid_yErr,ip_diffim_NaiveDipoleCentroid_pos_x,ip_diffim_NaiveDipoleCentroid_pos_y,ip_diffim_NaiveDipoleCentroid_pos_xErr,ip_diffim_NaiveDipoleCentroid_pos_yErr,ip_diffim_NaiveDipoleCentroid_neg_x,ip_diffim_NaiveDipoleCentroid_neg_y,ip_diffim_NaiveDipoleCentroid_neg_xErr,ip_diffim_NaiveDipoleCentroid_neg_yErr,base_SdssShape_xx,base_SdssShape_yy,base_SdssShape_xy,base_SdssShape_xxErr,base_SdssShape_yyErr,base_SdssShape_xyErr,base_SdssShape_x,base_SdssShape_y,base_SdssShape_instFlux,base_SdssShape_instFluxErr,base_SdssShape_psf_xx,base_SdssShape_psf_yy,base_SdssShape_psf_xy,base_SdssShape_instFlux_xx_Cov,base_SdssShape_instFlux_yy_Cov,base_SdssShape_instFlux_xy_Cov,base_CircularApertureFlux_3_0_instFlux,base_CircularApertureFlux_3_0_instFluxErr,base_CircularApertureFlux_4_5_instFlux,base_CircularApertureFlux_4_5_instFluxErr,base_CircularApertureFlux_6_0_instFlux,base_CircularApertureFlux_6_0_instFluxErr,base_CircularApertureFlux_9_0_instFlux,base_CircularApertureFlux_9_0_instFluxErr,base_CircularApertureFlux_12_0_instFlux,base_CircularApertureFlux_12_0_instFluxErr,base_CircularApertureFlux_17_0_instFlux,base_CircularApertureFlux_17_0_instFluxErr,base_CircularApertureFlux_25_0_instFlux,base_CircularApertureFlux_25_0_instFluxErr,base_CircularApertureFlux_35_0_instFlux,base_CircularApertureFlux_35_0_instFluxErr,base_CircularApertureFlux_50_0_instFlux,base_CircularApertureFlux_50_0_instFluxErr,base_CircularApertureFlux_70_0_instFlux,base_CircularApertureFlux_70_0_instFluxErr,base_GaussianFlux_instFlux,base_GaussianFlux_instFluxErr,base_PeakLikelihoodFlux_instFlux,base_PeakLikelihoodFlux_instFluxErr,base_PsfFlux_instFlux,base_PsfFlux_instFluxErr,base_PsfFlux_area,ip_diffim_NaiveDipoleFlux_pos_instFlux,ip_diffim_NaiveDipoleFlux_pos_instFluxErr,ip_diffim_NaiveDipoleFlux_neg_instFlux,ip_diffim_NaiveDipoleFlux_neg_instFluxErr,ip_diffim_NaiveDipoleFlux_npos,ip_diffim_NaiveDipoleFlux_nneg,ip_diffim_PsfDipoleFlux_pos_instFlux,ip_diffim_PsfDipoleFlux_pos_instFluxErr,ip_diffim_PsfDipoleFlux_neg_instFlux,ip_diffim_PsfDipoleFlux_neg_instFluxErr,ip_diffim_PsfDipoleFlux_chi2dof,ip_diffim_PsfDipoleFlux_pos_centroid_x,ip_diffim_PsfDipoleFlux_pos_centroid_y,ip_diffim_PsfDipoleFlux_pos_centroid_xErr,ip_diffim_PsfDipoleFlux_pos_centroid_yErr,ip_diffim_PsfDipoleFlux_neg_centroid_x,ip_diffim_PsfDipoleFlux_neg_centroid_y,ip_diffim_PsfDipoleFlux_neg_centroid_xErr,ip_diffim_PsfDipoleFlux_neg_centroid_yErr,ip_diffim_PsfDipoleFlux_centroid_x,ip_diffim_PsfDipoleFlux_centroid_y,ip_diffim_PsfDipoleFlux_centroid_xErr,ip_diffim_PsfDipoleFlux_centroid_yErr,ip_diffim_ClassificationDipole_value,ip_diffim_DipoleFit_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,ip_diffim_DipoleFit_pos_centroid_x,ip_diffim_DipoleFit_pos_centroid_y,ip_diffim_DipoleFit_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,ip_diffim_DipoleFit_neg_centroid_x,ip_diffim_DipoleFit_neg_centroid_y,ip_diffim_DipoleFit_centroid_x,ip_diffim_DipoleFit_centroid_y,ip_diffim_DipoleFit_instFlux,ip_diffim_DipoleFit_orientation,ip_diffim_DipoleFit_separation,ip_diffim_DipoleFit_chi2dof,ip_diffim_DipoleFit_signalToNoise,totFlux,totFluxErr,footprint,visit,filter,raft,detector
uint8,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,flo

Define an object ID

In [6]:
butler.getKeys('deepDiff_differenceExp')


{'visit': int,
 'filter': str,
 'raftName': str,
 'detectorName': str,
 'detector': int}

In [7]:
source_id = dict(
    visit=source['visit'],
    filter=source['filter'], 
    raftName=source['raft'],
    detector=source['detector'])

source_id

{'visit': 431306, 'filter': 'u', 'raftName': 'R10', 'detector': 28}

Create the cutout image. 

Todo: How to convert from degrees to pixels without a 'deepDiff_differenceExp_skyMap'? For not I'm arbitrarily choosing pixel (1000, 1000)

In [8]:
# Define object position
ra, dec = source['coord_ra'], source['coord_dec']
radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)

# Define image size
SideLength = 100
cutoutSize = afwGeom.ExtentI(SideLength, SideLength)

# Get source location in pixels
datasetType = 'deepDiff_differenceExp_sub'
# skymap = butler.get("%s_skyMap" % datasetType)

# Define cutour boundaries 
xy = afwGeom.PointI(1000, 1000)
bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

cutout_image = butler.get(datasetType, bbox=bbox, dataId=source_id)
cutout_image


NoResults: No locations for get: datasetType:deepDiff_differenceExp_sub dataId:DataId(initialdata={'visit': 431306, 'filter': 'u', 'raftName': 'R10', 'detector': 28, 'bbox': Box2I(minimum=Point2I(950, 950), dimensions=Extent2I(100, 100))}, tag=set())